In [ ]:
from __future__ import annotations

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

import qubex.pulse as qp
from qubex.simulator import (
    Control,
    Coupling,
    QuantumSimulator,
    QuantumSystem,
    Transmon,
)

In [ ]:
# units: ns, GHz
transmon_dimension = 3
control_frequency = 7.648
target_frequency = 8.275
anharmonicity = -0.333
relaxation_rate = 5e-5
dephasing_rate = 5e-5
coupling_strength = 0.01

qubits = [
    Transmon(
        label="Q01",
        dimension=transmon_dimension,
        frequency=control_frequency,
        anharmonicity=anharmonicity,
        relaxation_rate=relaxation_rate,
        dephasing_rate=dephasing_rate,
    ),
    Transmon(
        label="Q02",
        dimension=transmon_dimension,
        frequency=target_frequency,
        anharmonicity=anharmonicity,
        relaxation_rate=relaxation_rate,
        dephasing_rate=dephasing_rate,
    ),
]

couplings = [
    Coupling(
        pair=("Q01", "Q02"),
        strength=coupling_strength,
    ),
]

system = QuantumSystem(
    objects=qubits,
    couplings=couplings,
)

simulator = QuantumSimulator(system)

In [ ]:
target_qubit = qubits[0]

pulse_duration = 16  # ns

In [ ]:
def drag_pi_pulse(
    duration: float,
    beta: float,
) -> qp.Pulse:
    pulse = qp.Drag(
        duration=duration,
        amplitude=1,
        beta=beta,
    )
    norm_factor = np.pi / float(np.sum(np.abs(pulse.values) * pulse.SAMPLING_PERIOD))
    pulse = pulse.scaled(norm_factor)
    return pulse

In [ ]:
alpha = 2 * np.pi * target_qubit.anharmonicity
beta = -0.5 / alpha
pi_pulse = drag_pi_pulse(
    duration=pulse_duration,
    beta=beta,
)
pi_pulse.plot(divide_by_two_pi=True)

drag_amplitude = np.max(pi_pulse.real)
drag_amplitude

In [ ]:
result = simulator.mesolve(
    controls=[
        Control(
            target=target_qubit.label,
            frequency=target_qubit.frequency,
            waveform=pi_pulse,
        )
    ],
    initial_state=simulator.system.ground_state,
)
result.display_bloch_sphere(target_qubit.label)
result.show_last_population(target_qubit.label)

In [ ]:
results = []
beta_range = np.linspace(-1, 1, 101)
for beta in beta_range:
    alpha = 2 * np.pi * anharmonicity
    pulse = qp.Drag(
        duration=pulse_duration,
        amplitude=1,
        beta=beta,
    )
    norm_factor = np.pi / float(np.sum(np.abs(pulse.values) * pulse.SAMPLING_PERIOD))
    pulse = pulse.scaled(norm_factor)
    result = simulator.mesolve(
        controls=[
            Control(
                target=target_qubit.label,
                frequency=target_qubit.frequency,
                waveform=qp.PulseArray([
                    pulse,
                    pulse.scaled(-1),
                ])
            )
        ],
        initial_state=simulator.system.ground_state,
    )
    results.append(result)

In [ ]:
results[50].display_bloch_sphere(target_qubit.label)
results[60].display_bloch_sphere(target_qubit.label)
results[70].display_bloch_sphere(target_qubit.label)

In [ ]:
from qubex.analysis import plot_xy

e_x = np.array(
    [result.get_bloch_vectors(target_qubit.label)[-1][0] for result in results]
)

plot_xy(beta_range, e_x)

In [ ]:
from scipy.optimize import root_scalar

e_x_fine = lambda x: np.interp(x, beta_range, e_x)

result = root_scalar(e_x_fine, bracket=[beta_range[0], beta_range[-1]])
drag_beta = result.root
drag_beta

In [ ]:
def drag_objective_func(x):
    amplitude, beta = x
    pulse = qp.Drag(
        duration=pulse_duration,
        amplitude=amplitude,
        beta=beta,
    )
    result = simulator.mesolve(
        controls=[
            Control(
                target=target_qubit.label,
                frequency=target_qubit.frequency,
                waveform=qp.PulseArray(
                    [pulse] * 3,
                )
            )
        ],
        initial_state=simulator.system.ground_state,
    )
    state = result.get_bloch_vectors(target_qubit.label)[-1]
    target = np.array([0, 0, -1])
    return np.linalg.norm(state - target)

In [ ]:
drag_objective_func([drag_amplitude, drag_beta])

In [ ]:
import cma

initial_guess = [
    drag_amplitude,
    drag_beta,
]

es = cma.CMAEvolutionStrategy(
    initial_guess,
    0.01,
    {
        "seed": 42,
        "ftarget": 1e-6,
    },
)

es.optimize(drag_objective_func)

es.result.xbest

In [ ]:
drag_objective_func(es.result.xbest)

In [ ]:
pi_pulse = qp.Drag(
    duration=pulse_duration,
    amplitude=es.result.xbest[0],
    beta=es.result.xbest[1],
)
pi_pulse.plot(divide_by_two_pi=True)

In [ ]:
result = simulator.mesolve(
    controls=[
        Control(
            target=target_qubit.label,
            frequency=target_qubit.frequency,
            waveform=qp.PulseArray(
                [pi_pulse] * 1,
            ),
        )
    ],
    initial_state=simulator.system.ground_state,
)
result.display_bloch_sphere(target_qubit.label)
result.show_last_population(target_qubit.label)

In [ ]:
result = simulator.mesolve(
    controls=[
        Control(
            target=target_qubit.label,
            frequency=target_qubit.frequency,
            waveform=qp.PulseArray(
                [pi_pulse] * 2,
            ),
        )
    ],
    initial_state=simulator.system.ground_state,
)
result.display_bloch_sphere(target_qubit.label)
result.show_last_population(target_qubit.label)

In [ ]:
result = simulator.mesolve(
    controls=[
        Control(
            target=target_qubit.label,
            frequency=target_qubit.frequency,
            waveform=qp.PulseArray(
                [pi_pulse] * 10,
            ),
        )
    ],
    initial_state=simulator.system.ground_state,
)
result.display_bloch_sphere(target_qubit.label)
result.show_last_population(target_qubit.label)